In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%cp -R /kaggle/input/isic-2017-640x640/datasets /kaggle/working/datasets

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U pycocotools
!pip install -qr yolov5/requirements.txt  # install dependencies

In [ ]:
# Create the .yaml which controls the directories passed into the model

f = open("dataset.yaml", "a")
f.truncate(0) # clear file
f.write("test: /kaggle/working/datasets/Test/\n")
f.write("train: /kaggle/working/datasets/Train/\n")
f.write("val: /kaggle/working/datasets/Validate/\n")
f.write("nc: 2\n")
f.write("names: ['Not Melanoma', 'Melanoma']\n")
f.close()

In [ ]:
#%cd ..

In [ ]:
%cd /kaggle/working/yolov5

In [ ]:
#!pip install wandb

In [ ]:
# 20 Epochs Test
#!python train.py --img 640 --batch 16 --epochs 20 --data ../dataset.yaml --weights yolov5s.pt --device 0
#!python -m torch.distributed.run --nproc_per_node 2 train.py --img 640 --batch 16 --epochs 20 --data ../dataset.yaml --weights yolov5m.pt --device 0,1

In [ ]:
# 300 Epochs on YOLOv5s
!python train.py --img 640 --batch 16 --epochs 300 --data ../dataset.yaml --weights yolov5s.pt --device 0

In [ ]:
# 300 Epochs on YOLOv5m
!python train.py --img 640 --batch 16 --epochs 300 --data ../dataset.yaml --weights yolov5m.pt --device 0

In [ ]:
# Attempt to detect objects
!python detect.py --weights runs/train/exp4/weights/best.pt --img 640 --conf 0.4 --source /kaggle/working/datasets/Test/images

In [ ]:
# Show example output
from IPython.display import display
from PIL import Image

image_path = "runs/detect/exp/ISIC_0016065_resized_640.jpg"
display(Image.open(image_path))

In [ ]:
# Find Testing Accuracy
!python val.py --task test --data ../dataset.yaml --weights 'runs/train/exp4/weights/best.pt' --iou 0.8

In [ ]:
# FIND IOU
import torch
import numpy as np
import os
from PIL import Image

def get_IOU(box_a, box_b):
    if (box_a[0] - box_a[2]/2 > box_b[0] - box_b[2]/2):
        temp = box_a
        box_a = box_b
        box_b = temp
    x_left = max(box_a[0] - box_a[2]/2, box_b[0] - box_b[2]/2)
    y_top = max(box_a[1] - box_a[3]/2, box_b[1] - box_b[3]/2)
    x_right = min(box_a[0] + box_a[2]/2, box_b[0] + box_b[2]/2)
    y_bottom = min(box_a[1] + box_a[3]/2, box_b[1] + box_b[3]/2)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    intersect_area = (x_right - x_left) * (y_bottom - y_top)

    box_a_area = (box_a[2] * box_a[3])
    box_b_area = (box_b[2] * box_b[3])

    iou = intersect_area / float(box_a_area + box_b_area - intersect_area)

    return iou

def calculate_Image_IOU(test_image_path, label_path):
    # Import model
    model = torch.hub.load('ultralytics/yolov5', 'custom', '/kaggle/working/yolov5/runs/train/exp5/weights/best.pt')
    iou_list = []
    for dirname, _, filenames in os.walk(test_image_path):
        for filename in filenames:
            label_filename = filename.replace('.jpg', '.txt')
            file_path = os.path.join(dirname, filename)
            img = Image.open(file_path)
            prediction = model(img)
            prediction = prediction.pandas().xywh[0]

            # If no bounding boxes
            if not prediction.empty:
                center_x = prediction.loc[0]["xcenter"]/np.array(img).shape[0]
                center_y = prediction.loc[0]["ycenter"]/np.array(img).shape[1]
                width = prediction.loc[0]["width"]/np.array(img).shape[0]
                height = prediction.loc[0]["height"]/np.array(img).shape[1]
                #print(center_x, center_y, width, height)
                # Open label .txt
                label_val = open(label_path + label_filename).read().split()

                # Convert center coords to left most and top most
                #label_min_x = float(label_val[1]) - float(label_val[3])/2
                #label_min_y = float(label_val[2]) - float(label_val[4])/2

                iou = get_IOU([center_x, center_y, width, height], [float(label_val[1]), float(label_val[2]), float(label_val[3]), float(label_val[4])])
                iou_list.append(iou)

    return min(iou_list), np.average(iou_list)

min_iou, avg_iou = calculate_Image_IOU("/kaggle/working/datasets/Test/images/", "/kaggle/working/datasets/Test/labels/")
print(f'Min: {min_iou}\n')
print(f'Avg: {avg_iou}\n')

In [ ]:
# Run this first when running any of the yolov5 functions (such as val.py, detect.py etc)
%cd /kaggle/working/yolov5

In [ ]:
%ls
%cd runs

In [ ]:
%rm -r /kaggle/working/runs

In [ ]:
# List and link all items in /kaggle/working directory so it is downloadable
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLinks
FileLinks(r'yolov5')